In [349]:
# modules we'll use
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
from IPython.core.display import HTML

# read in our data
data = pd.read_csv(r'C:\\Users\\smsen\\Downloads\\BITS\\Course 2 - Feature Engineering\\Assignment - I\\bollywood.csv') # Reading CSV Input file and storing as pandas dataframe object
data1 = pd.read_csv(r'C:\\Users\\smsen\\Downloads\\BITS\\Course 2 - Feature Engineering\\Assignment - I\\bollywood - 2.csv') # Reading CSV Input file and storing as pandas dataframe object

In [350]:
print (data.shape)
print (data1.shape)

(51, 11)
(99, 10)


In [351]:
data.head(1)

,No,RelDate,MovieName,ReleaseTime,Genre,Budget,BoxOfficeCollection,YoutubeViews,YoutubeLikes,YoutubeDislikes,YoutubeLikeDislikes
0,1,Apr-18-2014,2 States,3,RomanceGenre,36Cr,104,8576361,26622.0,2527.0,29149


In [352]:
data1.head(1)

,Movie_ID,Release_Date,Movie_Name,Release_Time,Genre,BudgetCr,BoxOfficeCollectionCr,Youtube_Views,Youtube_Likes,Youtube_Dislikes
0,51,4-Jul-14,Lekar Hum Deewana Dil,N,Romance,16,2.5,3998331,2307,497


# Identify and rectify the eight prominent data quality issues present in the given datasets.

### Since the dataframe 2 is missing total Youtube likes and dislikes, create that variable in data1

In [353]:
sum_column = data1["Youtube_Likes"] + data1["Youtube_Dislikes"]
data1["YoutubeLikeDislikes"] = sum_column
data1.head(1)

,Movie_ID,Release_Date,Movie_Name,Release_Time,Genre,BudgetCr,BoxOfficeCollectionCr,Youtube_Views,Youtube_Likes,Youtube_Dislikes,YoutubeLikeDislikes
0,51,4-Jul-14,Lekar Hum Deewana Dil,N,Romance,16,2.5,3998331,2307,497,2804


In [354]:
#Rename dataframes
data.rename(columns={'No':'Movie_ID','RelDate':'Release_Date','MovieName':'Movie_Name','ReleaseTime':'Release_Time','YoutubeViews':'Youtube_Views','YoutubeLikes':'Youtube_Likes','YoutubeDislikes':'Youtube_Dislikes'}, inplace=True)

In [355]:
data1.rename(columns={'BudgetCr':'Budget','BoxOfficeCollectionCr':'BoxOfficeCollection'}, inplace=True)

In [356]:
#Concatenate both the datasets
concat=pd.concat([data,data1])

## Handling null values in youtube likes, dislikes

In [357]:
# we can quantify the total number of missing values using
# the isnull method plus the sum method on the dataframe
concat.isnull().sum()

Movie_ID               0
Release_Date           0
Movie_Name             0
Release_Time           0
Genre                  0
Budget                 0
BoxOfficeCollection    0
Youtube_Views          0
Youtube_Likes          1
Youtube_Dislikes       3
YoutubeLikeDislikes    0
dtype: int64

In [358]:
# alternatively, we can use the mean method after isnull to visualise the percentage of missing values for each variable
concat.isnull().mean()

Movie_ID               0.000000
Release_Date           0.000000
Movie_Name             0.000000
Release_Time           0.000000
Genre                  0.000000
Budget                 0.000000
BoxOfficeCollection    0.000000
Youtube_Views          0.000000
Youtube_Likes          0.006667
Youtube_Dislikes       0.020000
YoutubeLikeDislikes    0.000000
dtype: float64

There are missing data in the variables Youtube_Likes (0.06% missing) and Youtube_Likes (0.2% missing)

In [359]:
#As null values are found in YoutubeLikes and YoutubeDislikes, replace them with mean values
Avg_Youtube_Likes = concat['Youtube_Likes'].astype('float').mean(axis=0)
concat['Youtube_Likes'].replace(np.nan, Avg_Youtube_Likes, inplace= True)

In [360]:
Avg_Youtube_Dislikes = concat['Youtube_Dislikes'].astype('float').mean(axis=0)
concat['Youtube_Dislikes'].replace(np.nan, Avg_Youtube_Dislikes, inplace= True)

## Fix Genre, Budget & BoxOfficeCollection data

In [361]:
concat['Genre'] = concat.Genre.str.replace('Genre','') #Since Genre has invalid values, fix them by removing Genre from value and also strip each value
concat['Budget'] = concat['Budget'].replace(to_replace=r'Cr', value='', regex=True) #Since Budget has extra string 'Cr', fix them by removing 'Cr' from value
concat['BoxOfficeCollection'] = concat['BoxOfficeCollection'].replace(to_replace=r'Cr', value='', regex=True) #Since Budget has extra string 'Cr', fix them by removing 'Cr' from value

In [362]:
concat['Youtube_Likes'] = concat['Youtube_Likes'].astype(int)
concat['Youtube_Dislikes'] = concat['Youtube_Dislikes'].astype(int)
concat['Budget'] = concat['Budget'].astype(int)
concat['BoxOfficeCollection'] = concat['BoxOfficeCollection'].astype(float)

## Fix Invalid Release Time

In [363]:
#Since release time is text, replace it with number
concat['Release_Time'].replace('FS',1,inplace=True)
concat['Release_Time'].replace('HS',2,inplace=True)
concat['Release_Time'].replace('LW',3,inplace=True)
concat['Release_Time'].replace('N',4,inplace=True)

In [364]:
concat.Release_Time.value_counts()

4     99
2     18
1     17
3     15
44     1
Name: Release_Time, dtype: int64

In [365]:
#As the release_time has invalid value, replace it with correct value
concat.at[4, 'Release_Time' ]=4

## Fix Invalid Release Date

In [366]:
concat[concat.Release_Date.isnull()]

,Movie_ID,Release_Date,Movie_Name,Release_Time,Genre,Budget,BoxOfficeCollection,Youtube_Views,Youtube_Likes,Youtube_Dislikes,YoutubeLikeDislikes


In [367]:
concat.at[6, 'Release_Date' ]='Sep-19-2014'
concat.at[12, 'Release_Date' ]='Jun-27-2014'
concat.at[17, 'Release_Date' ]='Jun-6-2014'
concat.at[34, 'Release_Date' ]='Feb-21-2014'

In [368]:
#concat['Release_Date'] = pd.to_datetime(concat.Release_Date) #Since date columns are treated as string, let us convert them to date
concat['Release_Date'] = pd.to_datetime(concat['Release_Date'])

In [369]:
#Check if the right date format is set
concat.head(60)

,Movie_ID,Release_Date,Movie_Name,Release_Time,Genre,Budget,BoxOfficeCollection,Youtube_Views,Youtube_Likes,Youtube_Dislikes,YoutubeLikeDislikes
0,1,2014-04-18,2 States,3,Romance,36,104.00,8576361,26622,2527,29149
1,2,2013-01-04,Table No. 21,4,Thriller,10,12.00,1087320,1129,137,1266
2,3,2014-07-18,Amit Sahni Ki List,4,Comedy,10,4.00,572336,586,54,640
3,4,2013-01-04,Rajdhani Express,4,Drama,7,383.00,42626,86,19,105
4,5,2014-07-04,Bobby Jasoos,4,Comedy,18,10.80,3113427,4512,1224,5736
5,6,2014-05-30,Citylights,2,Drama,7,35.00,1076591,1806,84,1890
6,7,2014-09-19,Daawat-E-Ishq,4,Comedy,30,24.60,3905050,8315,1373,9688
7,8,2013-01-11,Matru Ki Bijlee Ka Mandola,4,Comedy,33,40.00,2435283,4326,647,4973
8,9,2014-01-10,Dedh Ishqiya,3,Comedy,31,27.00,2333067,2436,591,3027
9,10,2013-01-11,Gangoobai,4,Drama,2,0.01,4354,1,1221,2


## Find if any duplicate Movies

In [370]:
# to find out is there any duplicate by identity column Movie_ID
concat[concat.duplicated(['Movie_ID'])]

,Movie_ID,Release_Date,Movie_Name,Release_Time,Genre,Budget,BoxOfficeCollection,Youtube_Views,Youtube_Likes,Youtube_Dislikes,YoutubeLikeDislikes
0,51,2014-07-04,Lekar Hum Deewana Dil,4,Romance,16,2.5,3998331,2307,497,2804


In [371]:
# to find out is there any duplicate by column Movie_Name
concat[concat.duplicated(['Movie_Name'])]

,Movie_ID,Release_Date,Movie_Name,Release_Time,Genre,Budget,BoxOfficeCollection,Youtube_Views,Youtube_Likes,Youtube_Dislikes,YoutubeLikeDislikes
0,51,2014-07-04,Lekar Hum Deewana Dil,4,Romance,16,2.5,3998331,2307,497,2804


In [372]:
# Movie has two different release_dates and external source says the release date with index 51 is correct.
# so drop row with index = 50 
concat.drop_duplicates(subset ="Movie_ID", keep = 'first', inplace = True) 

In [373]:
display(HTML(concat.to_html()))

,Movie_ID,Release_Date,Movie_Name,Release_Time,Genre,Budget,BoxOfficeCollection,Youtube_Views,Youtube_Likes,Youtube_Dislikes,YoutubeLikeDislikes
0,1,2014-04-18,2 States,3,Romance,36,104.00,8576361,26622,2527,29149
1,2,2013-01-04,Table No. 21,4,Thriller,10,12.00,1087320,1129,137,1266
2,3,2014-07-18,Amit Sahni Ki List,4,Comedy,10,4.00,572336,586,54,640
3,4,2013-01-04,Rajdhani Express,4,Drama,7,383.00,42626,86,19,105
4,5,2014-07-04,Bobby Jasoos,4,Comedy,18,10.80,3113427,4512,1224,5736
5,6,2014-05-30,Citylights,2,Drama,7,35.00,1076591,1806,84,1890
6,7,2014-09-19,Daawat-E-Ishq,4,Comedy,30,24.60,3905050,8315,1373,9688
7,8,2013-01-11,Matru Ki Bijlee Ka Mandola,4,Comedy,33,40.00,2435283,4326,647,4973
8,9,2014-01-10,Dedh Ishqiya,3,Comedy,31,27.00,2333067,2436,591,3027
9,10,2013-01-11,Gangoobai,4,Drama,2,0.01,4354,1,1221,2


# How many records are present in the dataset? Print the metadata information of dataset

In [374]:
print("No. of record present in dataset  %d "% concat.shape[0])

No. of record present in dataset  149 


In [375]:
concat.describe()

,Movie_ID,Release_Time,Budget,BoxOfficeCollection,Youtube_Views,Youtube_Likes,Youtube_Dislikes,YoutubeLikeDislikes
count,149.000000,149.000000,149.000000,149.000000,1.490000e+02,149.000000,149.000000,149.000000
mean,75.000000,3.315436,29.442953,58.235369,3.337920e+06,7930.503356,1226.489933,9085.355705
std,43.156691,1.072300,28.237981,98.111494,3.504407e+06,12731.484451,1847.442175,14372.048371
min,1.000000,1.000000,2.000000,0.010000,4.354000e+03,1.000000,1.000000,2.000000
25%,38.000000,3.000000,11.000000,10.000000,1.076591e+06,1385.000000,224.000000,1637.000000
50%,75.000000,4.000000,21.000000,28.200000,2.375050e+06,4289.000000,619.000000,4913.000000
75%,112.000000,4.000000,35.000000,58.000000,4.550051e+06,9100.000000,1419.000000,10486.000000
max,149.000000,4.000000,150.000000,735.000000,2.317107e+07,101275.000000,11888.000000,111341.000000


In [376]:
concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 98
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Movie_ID             149 non-null    int64         
 1   Release_Date         149 non-null    datetime64[ns]
 2   Movie_Name           149 non-null    object        
 3   Release_Time         149 non-null    int64         
 4   Genre                149 non-null    object        
 5   Budget               149 non-null    int32         
 6   BoxOfficeCollection  149 non-null    float64       
 7   Youtube_Views        149 non-null    int64         
 8   Youtube_Likes        149 non-null    int32         
 9   Youtube_Dislikes     149 non-null    int32         
 10  YoutubeLikeDislikes  149 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(4), object(2)
memory usage: 12.2+ KB


# How many movies in each genre got released in different release times

In [377]:
concat.groupby(['Genre','Release_Time']).sum() #Grouping 'Genre' and 'Release time' by using sum fucntion

Movie_ID  Budget  BoxOfficeCollection  Youtube_Views  \
Genre     Release_Time                                                         
 Drama    1                   65      39                91.00        8224700   
          2                   86      25                45.50        2333660   
          3                   27     125               383.00        6395202   
          4                  102      93               592.21        7807242   
 Drama    1                  257      56                85.00        7613220   
          2                  280     100               758.75       14218298   
          4                 1780     409               729.95       46933210   
Action    1                  357     278               717.00       43610929   
          2                  235     208               251.00       12174200   
          3                  272     187               239.20       13989483   
          4                  904     690               991.34       66601214   
Action    4                   14       4                 1.80         235767   
Comedy    1                  369      93               404.00        1945982   
          2                  225     102                87.30       14428074   
          3                  352     200               436.50       15561095   
          4                 1694     515               732.34       59377878   
Romance   1                  373      74                74.00       12256792   
          2                  114      66               171.60       10061506   
          3                  298      95               201.13       12543091   
          4                 1070     394               874.30       52021174   
Thriller  1                  334     151               151.10        7273942   
          2                  100      16                29.00        3707889   
          3                  115      29                53.00        3049275   
          4                 1712     358               399.40       65418321   
Thriller  3                   17      60               162.00        7641333   
          4                   23      20                15.65        1926590   

                        Youtube_Likes  Youtube_Dislikes  YoutubeLikeDislikes  
Genre     Release_Time                                                        
 Drama    1                     24627              2632                27259  
          2                      5201               242                 5443  
          3                     37585             10269                47854  
          4                     22630              3108                24518  
 Drama    1                     26132              3792                29924  
          2                     46398              5884                52282  
          4                    116504             11565               128069  
Action    1                    144477             19064               163541  
          2                     27995              4677                32672  
          3                     33164              4383                37547  
          4                    196045             23519               219564  
Action    4                       192               164                  356  
Comedy    1                      7996              1604                 9600  
          2                     17149              3017                12274  
          3                     29803             11958                41761  
          4                    125194             20376               144482  
Romance   1                     25344              5380                30724  
          2                     22569              3137                25706  
          3                     31939              3378                35317  
          4                     97389             14398               111787  
Thriller  1                     14464        

In [378]:
pd.crosstab(concat.Release_Time, concat.Genre, margins=True, margins_name="Total",rownames=['Release_Time'])

Genre,Drama,Drama,Action,Action,Comedy,Romance,Thriller,Thriller,Total
Release_Time,,,,,,,,,
1,2,2,3,0,3,3,4,0,17
2,3,3,3,0,5,3,1,0,18
3,1,0,3,0,5,4,1,1,15
4,5,19,14,1,23,15,20,2,99
Total,11,24,23,1,36,25,26,3,149


# Which month of the year, maximum movie releases are seen

In [379]:
gkk = concat.groupby(concat['Release_Date'].dt.to_period('M')).size().reset_index(name='movie_count')

In [380]:
concat1 = concat.groupby(concat['Release_Date'].dt.strftime('%B'))['Movie_ID'].count().sort_values(ascending=False).reset_index(name='movie_count')

In [381]:
print("Maximum movies released in the month of %s with count %d" %(concat1.iloc[0,0], concat1.iloc[0,1]))

Maximum movies released in the month of January with count 20


# Which month of the year typically sees most releases of high budgeted movies, that is, movies with budget of 25 crore or more?                                                                          

In [382]:
concat2=concat[concat["Budget"]>=25]
concat2.groupby([concat2.Release_Date.dt.year.rename("Year"),concat2.Release_Date.dt.month.rename("Month")]).size().sort_values(ascending=False).reset_index(name="Count").head(8) #Grouping Year & Month to get the count

,Year,Month,Count
0,2013,7,5
1,2014,3,4
2,2013,9,4
3,2014,4,4
4,2013,8,4
5,2014,2,4
6,2015,1,3
7,2013,2,3


<font color=green>**'Month' column shows the month(denoted by number) which got highest number of releases during that year for movies with budget of 25 crore or more.**</font>

# Which are the top 10 movies with maximum return of investment (ROI)? Calculate ROI as (Box office collection – Budget) / Budget.                                                                      

In [383]:
concat["ROI"] = (concat['BoxOfficeCollection']-concat['Budget'])/concat['Budget'] #Calculating ROI and putting it in a new column

In [384]:
concat['ROI'].max() #Calculating max value from column ROI

53.714285714285715

In [385]:
display(HTML(concat.sort_values(["ROI"], ascending=False)[0:10][["Movie_Name","ROI"]].to_html(index=False))) #Top 10 movies with  highest ROI

Movie_Name,ROI
Rajdhani Express,53.714286
Aashiqui 2,8.166667
PK,7.647059
Grand Masti,7.514286
The Lunchbox,7.500000
Fukrey,6.240000
Mary Kom,5.933333
Shahid,5.666667
Humpty Sharma Ki Dulhania,5.500000
Bhaag Milkha Bhaag,4.466667
